#데이터 불러오기 및 라이브러리 import

In [ ]:
pip install ctgan

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 32.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, classification_report
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler
from ctgan import CTGAN
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/프로젝트 1/MBA.csv')
df.head(30)

,application_id,gender,international,gpa,major,race,gmat,work_exp,work_industry,admission
0,1,Female,False,3.30,Business,Asian,620,3,Financial Services,Admit
1,2,Male,False,3.28,Humanities,Black,680,5,Investment Management,NaN
2,3,Female,True,3.30,Business,NaN,710,5,Technology,Admit
3,4,Male,False,3.47,STEM,Black,690,6,Technology,NaN
4,5,Male,False,3.35,STEM,Hispanic,590,5,Consulting,NaN
5,6,Male,False,3.18,Business,White,610,6,Consulting,NaN
6,7,Female,False,2.93,STEM,Other,590,3,Technology,Admit
7,8,Male,True,3.02,Business,NaN,630,6,Financial Services,NaN
8,9,Male,False,3.24,Business,White,590,2,Nonprofit/Gov,NaN
9,10,Male,False,3.27,Humanities,Asian,690,3,Consulting,NaN


#데이터 전처리 - 결측치 처리

In [ ]:
df.isnull().sum()
# 'race' 컬럼에서 결측치를 'unknown'으로 채움
df['race'] = df['race'].fillna('unknown')

# 'admission' 컬럼에서 결측치를 'denied'로 채움
df['admission'] = df['admission'].fillna('denied')


#데이터 전처리 - 이상치 처리

In [ ]:
# 'admission' 컬럼에서 'Waitlist' 값을 'denied'로 변환
df['admission'] = df['admission'].apply(lambda x : 1 if x == 'Waitlist' or x =='denied' else 0)

# 'application_id' 컬럼은 필요 없으므로 제거
df = df.drop('application_id', axis = 1)

#데이터 전처리 - 데이터 증강

In [ ]:
# 데이터 준비 (admission 데이터 준비 완료)
# X와 y는 이미 준비되어 있고, 소수 클래스(0)에 대해 증강하려고 함

# 소수 클래스만 추출 (여기서는 '0'이 소수 클래스라고 가정)
minority_class_data = df[df['admission'] == 0]  # admission이 0인 데이터 추출
minority_class_data = minority_class_data.drop(columns=['admission'])  # 타겟 열 제외

# CTGAN 모델 정의 및 학습
ctgan = CTGAN(epochs=100)  # 학습할 epoch 수는 필요에 따라 조정 가능
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
ctgan.fit(minority_class_data, discrete_columns=categorical_cols)  # 범주형 컬럼 명시

# 새로운 데이터 생성 (예: 4800개의 소수 클래스 데이터 생성)
new_samples = ctgan.sample(4800)

# 생성된 데이터를 원래 소수 클래스의 레이블인 0으로 지정
new_samples['admission'] = 0

# 기존 데이터와 결합
df_augmented = pd.concat([df, new_samples], ignore_index=True)

#데이터 전처리 - 인코딩

In [ ]:
# LabelEncoder 객체 생성
le = LabelEncoder()

# 범주형 컬럼(object 타입)을 모두 찾아서 레이블 인코딩 수행
categorical_cols = df_augmented.select_dtypes(include=['object']).columns

# 각 범주형 컬럼에 Label Encoding 적용
for col in categorical_cols:
    df_augmented[col] = le.fit_transform(df_augmented[col])  # 인코딩 적용
    print(f"Classes (범주) for {col}: {le.classes_}")  # 원래 범주들 출력
    encoded_data = df_augmented[col]  # 인코딩된 값을 데이터프레임에 저장
    mapping = dict(zip(le.classes_, range(len(le.classes_))))  # 범주와 인코딩 값 매핑
    print(f"Mapping (범주 -> 인코딩 값) for {col}: {mapping}\n")

# 훈련  (증강된 데이터로)
X = df_augmented.drop('admission', axis=1)
y = df_augmented['admission']



Classes (범주) for gender: ['Female' 'Male']
Mapping (범주 -> 인코딩 값) for gender: {'Female': 0, 'Male': 1}

Classes (범주) for major: ['Business' 'Humanities' 'STEM']
Mapping (범주 -> 인코딩 값) for major: {'Business': 0, 'Humanities': 1, 'STEM': 2}

Classes (범주) for race: ['Asian' 'Black' 'Hispanic' 'Other' 'White' 'unknown']
Mapping (범주 -> 인코딩 값) for race: {'Asian': 0, 'Black': 1, 'Hispanic': 2, 'Other': 3, 'White': 4, 'unknown': 5}

Classes (범주) for work_industry: ['CPG' 'Consulting' 'Energy' 'Financial Services' 'Health Care'
 'Investment Banking' 'Investment Management' 'Media/Entertainment'
 'Nonprofit/Gov' 'Other' 'PE/VC' 'Real Estate' 'Retail' 'Technology']
Mapping (범주 -> 인코딩 값) for work_industry: {'CPG': 0, 'Consulting': 1, 'Energy': 2, 'Financial Services': 3, 'Health Care': 4, 'Investment Banking': 5, 'Investment Management': 6, 'Media/Entertainment': 7, 'Nonprofit/Gov': 8, 'Other': 9, 'PE/VC': 10, 'Real Estate': 11, 'Retail': 12, 'Technology': 13}



#데이터 전처리 - 스케일링

In [ ]:
# 데이터 스케일링
scaler = StandardScaler()
X_scaled_augmented = scaler.fit_transform(X)

#데이터 전처리 - 테스트셋 구분


In [ ]:
# 학습 및 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled_augmented, y, test_size=0.2, random_state=42)

#특성공학 - 카이제곱

In [ ]:
from scipy.stats import chi2_contingency

selected_columns = []

for element in df_augmented.columns:

    contingency_table = pd.crosstab(df_augmented[element], df_augmented['admission'])

    # 카이제곱 검정 수행
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    if p < 0.03:
        selected_columns.append(element)

    print(element)
    print("Chi-Square Statistic:", chi2)
    print("p-value:", p)
    print("Degrees of Freedom:", dof)
    print("")

gender
Chi-Square Statistic: 270.1925849306205
p-value: 1.0302349079189611e-60
Degrees of Freedom: 1

international
Chi-Square Statistic: 4066.766826983483
p-value: 0.0
Degrees of Freedom: 1

gpa
Chi-Square Statistic: 8236.899579390138
p-value: 1.3026604932856207e-174
Degrees of Freedom: 4900

major
Chi-Square Statistic: 8.42434455025787
p-value: 0.014814152944408326
Degrees of Freedom: 2

race
Chi-Square Statistic: 152.28210912596683
p-value: 4.361938782431616e-31
Degrees of Freedom: 5

gmat
Chi-Square Statistic: 7318.124215394642
p-value: 0.0
Degrees of Freedom: 242

work_exp
Chi-Square Statistic: 775.8699903079432
p-value: 3.261825388862487e-162
Degrees of Freedom: 8

work_industry
Chi-Square Statistic: 211.6423384239894
p-value: 5.512229306967556e-38
Degrees of Freedom: 13

admission
Chi-Square Statistic: 10989.994902306276
p-value: 0.0
Degrees of Freedom: 1



#특성공학 -랜덤포레스트

In [ ]:
# 랜덤 포레스트 모델 학습
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

# 특성 중요도 추출
importances = model.feature_importances_
print(len(importances))
# 특성 중요도를 데이터프레임으로 변환
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': importances})

# 중요도 순으로 정렬
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# 결과 출력
print(feature_importance_df)


8
         Feature  Importance
2            gpa    0.286088
1  international    0.249372
5           gmat    0.229750
4           race    0.091665
7  work_industry    0.055408
6       work_exp    0.047722
3          major    0.022032
0         gender    0.017963


#특성공학 - 전진제거법

In [ ]:
features = []
remaining_features = list(range(X.shape[1]))
best_score = 0

while remaining_features:
    scores = []
    for feature in remaining_features:
        # 현재 선택된 특성 + 새 특성
        selected_features = features + [feature]
        model = LogisticRegression(max_iter=200)
        model.fit(X_train[:, selected_features], y_train)
        predictions = model.predict(X_test[:, selected_features])
        score = f1_score(y_test, predictions)
        scores.append((score, feature))

    # 가장 성능이 좋은 특성 선택
    scores.sort(reverse=True)
    best_new_score, best_new_feature = scores[0]

    if best_new_score > best_score:
        best_score = best_new_score
        features.append(best_new_feature)
        remaining_features.remove(best_new_feature)
    else:
        break


print(f"최고 높은 점수: {best_score}")
for element in features:
    print(f"선택된 특성: {df.columns[element]}")

최고 높은 점수: 0.8755244755244755
선택된 특성: international
선택된 특성: gpa
선택된 특성: gmat
선택된 특성: race
선택된 특성: gender


#특성공학 - 라쏘

In [ ]:
lasso = Lasso(alpha=0.01)
lasso.fit(X_train, y_train)

lasso_coef = pd.Series(lasso.coef_, index=X.columns)
selected_features = lasso_coef[lasso_coef != 0].index

print(f"Selected Features by Lasso: {selected_features}")


Selected Features by Lasso: Index(['gender', 'international', 'gpa', 'race', 'gmat', 'work_exp'], dtype='object')


# 특성공학 - 특성 선택완료

1. gmat
2. gpa
3. work_exp

In [ ]:
X = df_augmented.drop(['admission', 'major', 'gender', 'race', 'international', 'work_industry'], axis=1)
# X = df_augmented.drop('admission', axis=1)
y = df_augmented['admission']

# 데이터 스케일링
scaler = StandardScaler()
X_scaled_augmented = scaler.fit_transform(X)

# 학습 및 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled_augmented, y, test_size=0.2, random_state=42)

#모델별 성능 평가

In [ ]:
models = [
    # ("Logistic Regression", LogisticRegression(class_weight='balanced')),
    # ("K-Nearest Neighbors", KNeighborsClassifier(n_neighbors=3)),
    # ("Decision Tree", DecisionTreeClassifier(class_weight='balanced', random_state=42)),
    # ("Random Forest", RandomForestClassifier(class_weight='balanced', random_state=42)),
    # ("Support Vector Machine", SVC(probability=True)),
    ("XGBOOST", XGBClassifier(scale_pos_weight=10, random_state=42)),
    # ("GaussianNB", GaussianNB()),
    # ("GradientBoostingClassifier", GradientBoostingClassifier()),
    # ("AdaBoostClassifier", AdaBoostClassifier())
]

for model_name, model in models:
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print(f"\nModel: {model_name}")
    print(f"Accuracy: {accuracy * 100:.2f}%")

    print("Classification Report:")
    print(classification_report(y_test, y_pred))

    if hasattr(model, "predict_proba"):
        y_probs = model.predict_proba(X_test)[:, 1]
        roc_auc = roc_auc_score(y_test, y_probs)
        print(f"ROC-AUC: {roc_auc:.2f}")


Model: Logistic Regression
Accuracy: 87.81%
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.87      0.88      1127
           1       0.87      0.88      0.88      1072

    accuracy                           0.88      2199
   macro avg       0.88      0.88      0.88      2199
weighted avg       0.88      0.88      0.88      2199

ROC-AUC: 0.94

Model: K-Nearest Neighbors
Accuracy: 86.49%
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.85      0.87      1127
           1       0.85      0.88      0.86      1072

    accuracy                           0.86      2199
   macro avg       0.87      0.87      0.86      2199
weighted avg       0.87      0.86      0.86      2199

ROC-AUC: 0.92

Model: Decision Tree
Accuracy: 86.36%
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.87      0.87      1127
           

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(



Model: AdaBoostClassifier
Accuracy: 88.49%
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.85      0.88      1127
           1       0.85      0.92      0.89      1072

    accuracy                           0.88      2199
   macro avg       0.89      0.89      0.88      2199
weighted avg       0.89      0.88      0.88      2199

ROC-AUC: 0.96


#XGBOOST하이퍼 파라미터 튜닝 및 평가

In [ ]:
from sklearn.model_selection import GridSearchCV

# 하이퍼파라미터 범위 설정
param_grid = {
    'n_estimators': [100, 200, 300],      # 트리 개수
    'learning_rate': [0.01, 0.1, 0.3],    # 학습률 (스텝 크기)
    'max_depth': [3, 5, 7],               # 트리의 최대 깊이
    'subsample': [0.8, 1.0],              # 데이터 샘플링 비율
    'colsample_bytree': [0.8, 1.0],       # 트리를 만들 때 사용할 컬럼의 샘플링 비율
    'gamma': [0, 1, 5],                   # 노드를 분할하기 위한 최소 손실 감소
    'reg_lambda': [1, 10],                # 가중치에 대한 L2 정규화 값
    'scale_pos_weight': [1, 10]           # 클래스 불균형을 조정하는 가중치
}

# XGBoost 분류기 초기화
xgb = XGBClassifier(random_state=42)

# 5겹 교차 검증을 사용하는 GridSearchCV 설정
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, scoring='roc_auc', cv=5, verbose=2, n_jobs=-1)

# GridSearchCV 실행 (학습)
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
print("최적의 하이퍼파라미터: ", grid_search.best_params_)

# 최적의 하이퍼파라미터로 모델 재학습
best_xgb = grid_search.best_estimator_

# 예측 및 평가
y_pred_best = best_xgb.predict(X_test)
accuracy_best = accuracy_score(y_test, y_pred_best)
print(f"최적 XGBoost 모델 정확도: {accuracy_best * 100:.2f}%")

print("분류 리포트:")
print(classification_report(y_test, y_pred_best))

# 최적 모델의 ROC-AUC 점수 계산
if hasattr(best_xgb, "predict_proba"):
    y_probs_best = best_xgb.predict_proba(X_test)[:, 1]
    roc_auc_best = roc_auc_score(y_test, y_probs_best)
    print(f"최적 모델 ROC-AUC: {roc_auc_best:.2f}")


Fitting 5 folds for each of 1296 candidates, totalling 6480 fits
최적의 하이퍼파라미터:  {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.3, 'max_depth': 3, 'n_estimators': 200, 'reg_lambda': 1, 'scale_pos_weight': 10, 'subsample': 1.0}
최적 XGBoost 모델 정확도: 87.86%
분류 리포트:
              precision    recall  f1-score   support

           0       1.00      0.76      0.87      1127
           1       0.80      1.00      0.89      1072

    accuracy                           0.88      2199
   macro avg       0.90      0.88      0.88      2199
weighted avg       0.90      0.88      0.88      2199

최적 모델 ROC-AUC: 0.96
